In [4]:
# Install required packages
!pip install numpy pandas scikit-learn nltk

import numpy as np
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random
import string

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load sample healthcare dataset
# In a real application, you would use a more comprehensive dataset
data = {
    'symptoms': [
        'headache fever', 'cough sore throat', 'stomach pain nausea',
        'chest pain shortness of breath', 'fatigue dizziness',
        'rash itching', 'joint pain swelling', 'back pain',
        'abdominal pain diarrhea', 'sneezing runny nose'
    ],
    'conditions': [
        'flu', 'common cold', 'food poisoning',
        'possible heart issues - seek immediate care', 'anemia or low blood pressure',
        'allergic reaction', 'arthritis', 'muscle strain',
        'stomach flu', 'allergies'
    ],
    'advice': [
        'rest, drink fluids, take acetaminophen',
        'rest, drink warm fluids, consider over-the-counter cold medicine',
        'stay hydrated, avoid solid foods for a few hours, rest',
        'THIS MAY BE SERIOUS - seek emergency medical attention immediately',
        'eat iron-rich foods, stay hydrated, rise slowly from sitting',
        'take antihistamines, avoid scratching, identify allergens',
        'apply warm compress, take anti-inflammatory medication',
        'apply ice pack, rest, gentle stretching when pain subsides',
        'BRAT diet (bananas, rice, applesauce, toast), stay hydrated',
        'take antihistamines, avoid allergens'
    ]
}

df = pd.DataFrame(data)

# Preprocessing
lemmatizer = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

def LemNormalize(text):
  return LemTokens(nltk.word_tokenize(text.lower().translate(str.maketrans('', '', string.punctuation))))

# TF-IDF Vectorizer
tfidf_vec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
tfidf = tfidf_vec.fit_transform(df['symptoms'])

# Chatbot function
def healthcare_chatbot(user_input):
    # Transform user input
    user_tfidf = tfidf_vec.transform([user_input])

    # Calculate similarity
    vals = cosine_similarity(user_tfidf, tfidf)
    idx = vals.argsort()[0][-1]
    flat = vals.flatten()
    flat.sort()
    score = flat[-1]

    if score < 0.2:  # Threshold for similarity
        return "I'm not sure I understand your symptoms. Could you please describe them in more detail?"
    else:
        response = f"Based on your symptoms, it might be: {df['conditions'][idx]}. \n\nRecommendation: {df['advice'][idx]}"
        if "emergency" in df['advice'][idx].lower() or "serious" in df['advice'][idx].lower():
            response += "\n\nTHIS MAY BE A MEDICAL EMERGENCY - PLEASE SEEK IMMEDIATE CARE!"
        return response

# Greetings and goodbyes
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up", "hey")
GREETING_RESPONSES = ["hi", "hey", "hi there", "hello", "I'm glad you're talking to me"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

# Main chat loop
print("Healthcare Chatbot: Hi, I'm your healthcare assistant. Describe your symptoms and I'll try to help. Type 'quit' to exit.")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        print("Healthcare Chatbot: Take care and feel better soon!")
        break

    if greeting(user_input) is not None:
        print("Healthcare Chatbot: " + greeting(user_input))
    else:
        print("Healthcare Chatbot: " + healthcare_chatbot(user_input))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Healthcare Chatbot: Hi, I'm your healthcare assistant. Describe your symptoms and I'll try to help. Type 'quit' to exit.
You: hello
Healthcare Chatbot: I'm glad you're talking to me
You: fever
Healthcare Chatbot: Based on your symptoms, it might be: flu. 

Recommendation: rest, drink fluids, take acetaminophen
You: cough
Healthcare Chatbot: Based on your symptoms, it might be: common cold. 

Recommendation: rest, drink warm fluids, consider over-the-counter cold medicine
You: quit
Healthcare Chatbot: Take care and feel better soon!
